In [ ]:
!pip install keytotext --upgrade
!sudo apt-get install git-lfs

# Read data
Read all cleaned data from *'./data/cleaned_data_final.csv'*.

In [1]:
import pandas as pd

file='./data/cleaned_data_final.csv'
df=pd.read_csv(file)
df = df.iloc[:,1:]
df=df.rename(columns={'generated_keywords': 'keywords', 'description': 'text'})[['keywords','text']]
df

,keywords,text
0,vast growing library educators crew,A crew of experienced educators helms our vast...
1,education process values beliefs facilitating,Education is the process of facilitating learn...
2,heard teachers strong building dozens,Education Week talked to dozens of principals ...
3,school like discipline methods environments,"Education, discipline that is concerned with m..."
4,point school integrated selective sharply,With New York sharply divided over gifted educ...
...,...,...
8011,cut loose crowds search undone,Cap 2 bottoms - Cut loose from the maddening c...
8012,crew takes weather edge cap,Cap 2 crew - This crew takes the edge off fick...
8013,use morning need times shell,All-time shell - No need to use that morning T...
8014,mud job shorts cargo bask,All-wear cargo shorts - All-Wear Cargo Shorts ...


##  Split data set

In [ ]:
from sklearn.model_selection import train_test_split
train_df, test_df= train_test_split(df, test_size=0.33, random_state=42)
len(train_df),len(test_df)


(5370, 2646)

# Train the model
To train the model, first we use the pretrained model *"t5-base-finetuned-common_gen"* of the keytotex library. Then using the new data set we retrain the model for our goal. 
The new trained model will be saved as *"modelv1"*

In [ ]:
from keytotext import trainer
preTrainedModel="mrm8488/t5-base-finetuned-common_gen"
model = trainer()
model.from_pretrained(model_name=preTrainedModel)
model.train(train_df=train_df[:], test_df=test_df[:], batch_size=2, max_epochs=3,use_gpu=True)
model.save_model( model_dir="modelv1")

Global seed set to 42
wandb: Currently logged in as: habibi. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:97: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=5)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  f"Setting `Trainer(progress_bar_refresh_rate={progress_bar_refresh_rate})` is deprecated in v1.5 and"
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainabl

Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/torchmetrics/utilities/prints.py:36: UserWarning: Torchmetrics v0.9 introduced a new argument class property called `full_state_update` that has
                not been set for this class (_ResultMetric). The property determines if `update` by
                default needs access to the full metric state. If this is not the case, significant speedups can be
                achieved and we recommend setting this to `False`.
                We provide an checking function
                `from torchmetrics.utilities import check_forward_no_full_state`
                that can be used to check if the `full_state_update=True` (old and potential slower behaviour,
                default for now) or if `full_state_update=False` can be used safely.
                
  warnings.warn(*args, **kwargs)


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

## Time to test the trained model
Set *use_gpu=Flase* if you do not gpu.

In [ ]:
from keytotext import trainer
model = trainer()
model.load_model( model_dir="modelv1",use_gpu=True)

#keywords=["ski", "mountain", "sky"]
keywords=['students',  'poverty', 'achievement']
print(model.predict(keywords,use_gpu=True))

params = {"do_sample":True, "num_beams":1, "early_stopping":True}    #decoding params
print(model.predict(keywords, **params))  #keywords


The achievement of all students in poverty is based on their education.
The achievement of students with poverty is a major challenge.


In [ ]:
!du -sh modelv1/*

4.0K	model/config.json
851M	model/pytorch_model.bin
4.0K	model/special_tokens_map.json
776K	model/spiece.model
4.0K	model/tokenizer_config.json
2.4M	model/tokenizer.json


## Now you can upload the model to api